In [45]:
import torch
import json
import requests
from PIL import Image
from io import BytesIO
from torchvision import models, transforms

In [51]:
class ImagePrediction:
    
    def __init__(self):
        self.model =  models.inception_v3(pretrained=True)
        self.model.eval()
        self.load_class_labels()
        
    
    def load_class_labels(self):
        try:
            with open("imagenet_class_index.json") as f:
                class_idx = json.load(f)
        except FileNotFoundError:
            url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
            response = requests.get(url)
            class_names = response.json()
            class_idx = {str(i): [str(i), name] for i, name in enumerate(class_names)}
            with open("imagenet_class_index.json", "w") as f:
                json.dump(class_idx,f)
        
    def preprocess_image(self,image):
        preprocess = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            
        ])
        
        return preprocess(image)
    
    
                
    def predict_image(self,image_path,top_k=1):
        if image_path.startswith(("http://","https://")):
            response = requests.get(image_path)
            image = Image.open(BytesIO(response.content))
        else:
            image = Image.open(image_path)
            
        if image.mode != "RGB":
            image = image.convert("RGB")
            
        input_tensor = self.preprocess_image(image)
        input_batch = input_tensor.unsqueeze(0)
        
        with torch.no_grad():
            output = self.model(input_batch)
        
        
        return np.argmax(output)
    
    def result(self,image):
        prediction = image_predicator.predict_image(image)
        #print(f"Prediction: {prediction}")

        return prediction, data[str(prediction.item())][1]
#else:
    

In [ ]:
image_predicator = ImagePrediction()
print(image_predicator.result("https://nypost.com/wp-content/uploads/sites/2/2016/07/hot-dog-2.jpg?quality=90&strip=all&w=1280"))

d:\software\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\software\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(tensor(934), 'hot dog')


: 

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import json
import requests
from io import BytesIO



class ImagePredicator:
    def __init__(self):
        pass
    
    def predict_image(self,image,top_k=1): 
        
        model = models.inception_v3(pretrained=True)
        model.eval()  # Set to evaluation mode

        # Create the preprocessing pipeline
        preprocess = transforms.Compose([
            transforms.Resize(299),  # Inception v3 requires 299x299 input
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        
        image = Image.open(image)
            
            # Convert image to RGB if it's not
        if image.mode != 'RGB':
            image = image.convert('RGB')
            
            # Preprocess the image
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0)  # Add batch dimension
            
            # If you have GPU
        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            model.to('cuda')
            
            # Perform inference
        with torch.no_grad():
            output = model(input_batch)
            
            # The output has unnormalized scores. To get probabilities, run a softmax on it
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
            
            # Get the top-k predictions
        top_probs, top_indices = torch.topk(probabilities, top_k)
            
            # Load the class labels
            # Download the mapping (only need to do this once)
        try:
            with open('imagenet_class_index.json') as f:
                class_idx = json.load(f)
        except FileNotFoundError:
            url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
            response = requests.get(url)
            class_names = response.json()
            class_idx = {str(i): [str(i), name] for i, name in enumerate(class_names)}
            with open('imagenet_class_index.json', 'w') as f:
                json.dump(class_idx, f)
            
            
        results = []
        for i in range(top_k):
            idx = top_indices[i].item()
            results.append({
                'class': class_idx[str(idx)][1],
                'probability': top_probs[i].item()
            })
            
        return results

       
image_path = "C:\\Users\\rppaw\\OneDrive\\Pictures\\Screenshots\\screen.png"  
i = ImagePredicator()
predictions = i.predict_image(image_path, top_k=5)
print(predictions[0]["class"])


tiger cat
